In [3]:
SEASON = '2012-13'

In [4]:
import pandas as pd
import myconstants as c
from scipy.stats import zscore

NUM_GAMES = c.NUM_GAMES[SEASON]
MIN_GAMES = NUM_GAMES * 0.60

p_df = pd.read_csv(f'../data/years/{SEASON}/clean/{SEASON}_Player_PerGame.csv')
t_df = pd.read_csv(f'../data/years/{SEASON}/clean/{SEASON}_Team_Standings.csv')

# join player per game and advanced stats
p_advanced_df = pd.read_csv(f'../data/years/{SEASON}/clean/{SEASON}_Player_Advanced.csv')
p_advanced_df = p_advanced_df.drop(columns=['Player', 'Tms', 'Pos', 'Age', 'G', 'MP'])
p_df = p_df.merge(p_advanced_df, on='PlayerID')

# populate team df with team codes
t_df['TeamCode'] = t_df['Team'].apply(lambda x: c.TEAM_CODE[x])

# average team wins per player
def average_wins(tms):
    tms_list = tms.split(',')
    tms_wins = []
    for t in tms_list:
        tms_wins.append(t_df.loc[t_df['TeamCode'] == t]['OverallW'].values[0])
    return sum(tms_wins) / len(tms_wins)
p_df['TmsWins'] = p_df['Tms'].apply(average_wins)

# players who haven't played enough games don't qualify
p_df = p_df.loc[p_df['G'] >= MIN_GAMES]

# ranking algorithm
p_df['PlayerScore'] = 1.5*zscore(p_df['PTS'] + p_df['TRB'] + p_df['AST'] + p_df['STL'] + p_df['BLK']) + zscore(p_df['WS']) + zscore(p_df['VORP']) + zscore(p_df['BPM']) + zscore(p_df['TmsWins'])
p_df = p_df.sort_values('PlayerScore', ascending=False)

# Move [PlayerScore] to the right of [Pos], drop PlayerID, index rows starting from 1, stylistic choice
col = p_df.pop('PlayerScore')
p_df.insert(p_df.columns.get_loc('Pos') + 1, col.name, col)
p_df = p_df.drop('PlayerID', axis=1)
p_df = p_df.reset_index(drop=True)
p_df.index = p_df.index + 1

p_df.head(50)

,Player,Tms,Pos,PlayerScore,Age,G,GS,MP,FG,FGA,...,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,TmsWins
1,LeBron James,MIA,PF,22.739939,28,76,76,37.9,10.1,17.8,...,30.2,14.6,4.7,19.3,0.322,9.3,2.4,11.7,9.9,66.0
2,Kevin Durant,OKC,SF,20.255482,24,81,81,38.5,9.0,17.7,...,29.8,13.6,5.3,18.9,0.291,7.6,1.7,9.3,8.9,60.0
3,Chris Paul,LAC,PG,14.495854,27,70,70,33.4,5.9,12.2,...,22.6,10.6,3.4,13.9,0.287,6.4,2.4,8.8,6.4,56.0
4,James Harden,HOU,SG,12.688779,23,78,78,38.3,7.5,17.1,...,29.0,9.8,3.0,12.8,0.206,5.2,0.5,5.6,5.8,45.0
5,Russell Westbrook,OKC,PG,12.042461,24,82,82,34.9,8.2,18.7,...,32.8,7.7,3.9,11.6,0.195,4.6,-0.2,4.4,4.6,60.0
6,Stephen Curry,GSW,PG,11.508789,24,78,78,38.2,8.0,17.8,...,26.4,8.4,2.8,11.2,0.180,5.3,0.1,5.4,5.6,47.0
7,Kobe Bryant*,LAL,SG,11.436125,34,78,78,38.6,9.5,20.4,...,31.9,8.4,2.6,10.9,0.174,4.7,-0.1,4.6,5.0,45.0
8,Marc Gasol,MEM,C,10.652326,28,80,80,35.0,5.4,10.9,...,19.2,6.1,5.4,11.5,0.197,2.1,3.1,5.2,5.1,56.0
9,Carmelo Anthony,NYK,PF,10.641631,28,67,67,37.0,10.0,22.2,...,35.6,7.5,2.0,9.5,0.184,6.0,-1.7,4.3,3.9,54.0
10,Dwyane Wade,MIA,SG,10.619121,31,69,69,34.7,8.2,15.8,...,29.5,6.1,3.5,9.6,0.192,3.7,0.7,4.4,3.8,66.0
